# FUNCIÓN DE TOKENIZACIÓN DE TEXTOS

Para hacer el análisis de texgtos es necesario tener cada palabra como registro, por ello se pasa a realziar un algoritmo que genera una tabla multi indice en donde cada registro contiene una palabra que está ligado a su texto y a su universidad. 

# PSEUDOCÓDIGO

Para entender mejor el procedimiento del código, se realizará un pseudocódigo más cercano al lenguaje humano.

Concetar a la base de datos<br>
Recoger columnas necesarias de la tabla<br>
Selecionar los textos<br>
hacer todos los caracteres minusculas<br>
tokenizar mediante algoritmo<br>
para cada palabra<br>
&emsp; identificar si es una stopword<br>
&emsp; si es una stopword <br>
&emsp; &emsp; clasificaren True<br>
&emsp; si no lo es<br>
&emsp; &emsp; clasificar en false<br>

&emsp; crear tabla<br>


# LIBRERÍAS


**__NLTK__**: Una de las librerías de Python más populares que se asocia a las palabrqas Natural Language Toolkit, que en su traducción al español significa algo así como la mochila de herramientas para el lenguaje natural. Está librería se encarga de que las máquinas entiendan el lenguaje humano y se provee a la librería con infinidad de herramientas para codificar programas para el procesamiento del lenguaje natural. De está librería se usará el TweetTokenizer, que nos permite individualizar cada palabra, signo o elemento y registrarlo en una fila individual creando un multiindex. También se ha usado el corpus para descargar una lista de stopwords en español.
, se puede consultar la documentación  __[aquí](https://www.nltk.org/)__.<br><br>

**SQL_ Alchemy**: Esta librería ayuda a generar una tabla de datos dentro de la base de datos seleccionada, la peculiaridad es que introduce los datos desde un dataframe de pandas en un proceso muy ágil, se puede consultar la documentación  __[aquí](https://www.sqlalchemy.org/)__.<br><br>

Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres
- **tqdm**: Enseña barra de progreso

Una vez ya se explica todo, se da comienzo con el código de la tokenización, importando las librerias y creando la conexión a la base de datos

In [ ]:
import mysql.connector
import codecs
import pandas as pd
from nltk.tokenize import TweetTokenizer
import nltk
#nltk.download('stopwords')
tweet_tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
import string
from tqdm import tqdm
from sqlalchemy import create_engine

codecs.register(lambda name: codecs.lookup('utf8') if name == 'utf8mb4' else None)

database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

APrimero, se crean dos funciones que serán útiles luego en el código

In [ ]:
def minusculas(x): #convierte el texto a minusculas
    try:
       x =  x.lower()
    except:
        pass
    return x
def tokeni(x):# tokeniza los textos
    try:
        x = x.split(x)
    except:
        pass
    return  x

Acto seguido, se ejecuta el código

In [ ]:
df = pd.read_sql("SELECT * FROM perfiles_linkedin",database)

df["Especialidades"] = df["Especialidades"].apply(lambda x: minusculas(x))
df["tokens_espanol"] = df["Especialidades"].apply(lambda x: tokeni(x))

orden = df.explode(column= "tokens_espanol")
palabrasstop = list(stopwords.words("spanish"))
palabrasstop.extend(list(string.punctuation))
palabrasstop.extend(["  ", " ", "\n","¡","¿"])

Se seleccionan los textos y se crea la lista de palabras stop.

In [ ]:
sino = []
for palabra in tqdm(orden["tokens_espanol"]):#para cada palaabra
    if type(palabra) != str:#si no es texto
            sino.append("True")
    elif palabra in palabrasstop:
        sino.append("True")
    elif palabra.isalpha() == False: #si no es alfabetino
        sino.append("True")
    else:
        sino.append("False")
orden["stop_espanol"] = sino#añadimos la nueva columna

Se clasifican las palabras en stopwords o no, para ahora crear la nueva tabla e isertarla en la base de datos.

In [ ]:

dataFrame   = pd.DataFrame(data=orden)           

tableName = "tokenizado_linkedin_especialidades"

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/TFG', pool_recycle=3600)

dbConnection    = sqlEngine.connect()

 

try:

    frame           = dataFrame.to_sql(tableName, dbConnection, if_exists='fail');

except ValueError as vx:

    print(vx)

except Exception as ex:   

    print(ex)

else:

    print("Table %s created successfully."%tableName);   

finally:

    dbConnection.close()

## Con este código se tokeniza todos los textos, cambiando la tabla y la columna

# Tiene una duración de 4 minutos para 1,2M de registros pero la máquina no soporta crear una tabla de 29M de registros